In [30]:
import oineus as oin
import numpy as np
import torch
import diode

def get_boundary_filtration(points, n_atoms_per_mol):
    points = np.asarray(points)
    simplices = diode.fill_alpha_shapes(points)
    fil = oin.Filtration_double([oin.Simplex_double(s[0], s[1]) for s in simplices])
    def is_multi(sigma):
        return len(set(v // n_atoms_per_mol for v in sigma.vertices)) >= 2
    fil = fil.subfiltration(is_multi)

    def assign_filtration_value(cell):
        parts = [v // n_atoms_per_mol for v in cell.vertices]
        n_parts = len(set(parts))
        p_agg = [np.array([0.0, 0.0, 0.0]) for _ in range(n_parts)]
        weights = [0 for _ in range(n_parts)]
        for i, p in enumerate(parts):
            p_agg[p] += points[cell.vertices[i]]
            weights[p] += 1
        bcs = [p_agg[i] / weights[i] for i in range(n_parts)]
        filtration_value = sum([np.linalg.norm(bcs[i] - bcs[j]) for i in range(n_parts) for j in range(i, n_parts)])
        cell.value = filtration_value
        return cell

    altered_cells = [assign_filtration_value(cell) for cell in fil.cells()]
    fil = oin.Filtration_double(altered_cells)
    return fil

In [31]:
def get_points(file):
    points = []
    with open(file) as f:
        for (i, line) in enumerate(f):
            if i >= 3:
                x, y, z = line.split()
                points.append(np.array([float(x), float(y), float(z)]))
    return points

In [32]:
points = get_points("../../../../Data/paper2/measure_paths/txts/6r7m/1/1.txt")
get_boundary_filtration(points, 1206)

Filtration(size = 1599, 
cells = [([1205, 1206], 1.45587),
([1205, 1209], 2.45578),
([1205, 1207], 2.45733),
([1205, 2407], 2.66753),
([1205, 1208], 2.71577),
([1205, 1210], 2.97469),
([1205, 2401], 3.62454),
([1205, 2403], 3.65939),
([1205, 2399], 3.79948),
([1205, 2402], 3.89931),
([1205, 2397], 4.04055),
([1205, 2400], 4.09067),
([1205, 2404], 4.12355),
([1205, 1671], 5.36349),
([1205, 1673], 7.19759),
([1205, 1674], 7.38621),
([517, 1401], 7.53835),
([517, 1367], 7.78713),
([518, 1401], 7.85226),
([517, 1400], 8.29718),
([511, 1400], 8.448),
([512, 1368], 8.51375),
([518, 1400], 8.51524),
([510, 1400], 8.57481),
([514, 1367], 8.63191),
([512, 1400], 8.66993),
([517, 1368], 8.95598),
([509, 1400], 8.95987),
([508, 1368], 9.00835),
([512, 1367], 9.04259),
([508, 1367], 9.12185),
([514, 1368], 9.37023),
([516, 1367], 9.50357),
([514, 1366], 9.77295),
([511, 1422], 10.1996),
([511, 1392], 10.2241),
([516, 1368], 10.3108),
([512, 1392], 10.4356),
([516, 1366], 10.4442),
([511, 1421], 10